<a href="https://colab.research.google.com/github/dineshdilip2/Dinesh-K/blob/main/Twitter_scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import snscrape.modules.twitter as tw
import pandas as pd
from pymongo import MongoClient
import streamlit as st
import csv 
import json


# Define function to scrape Twitter data and store in MongoDB

def scrape_twitter_data(keyword,start_date,end_date,tweet_limit,db):
    tweets =[]
    for tweet in tw.TwitterSearchScraper(f"{keyword} since:{start_date} untill:{end_date}").get_items():
        if len(tweets)==tweet_limit:
            break
        tweets.append({
            "date":tweet.date,
            "id":tweet.id,
            "url":tweet.url,
            "content":tweet.content,
            "user":tweet.user.username,
            "reply_count":tweet.replyCount,
            "retweet_count":tweet.retweetCount,
            "language":tweet.lang,
            "source":tweet.sourceLabel,
            "like_count":tweet.likeCount
         })
        
#Store the scraped data in Mongodb collection

    collection = db[keyword]
    collection.insert_one(tweets)
    st.success(f"{len(tweets)} tweets were scraped and saved to MongoDB.")
     
    return pd.Dataframe(tweets)

def export_data(data,filename,format):
    if format == "CSV":
        data.to_csv(filename,index=False)
        st.success(f"The data was exported to {filename} in CSV format.")
    elif format =="JSON":
        data.to_json(filename,orient= "records")
        st.success(f"The data was exported to {filename} in JSON format.")
    
#connnect to MongoDB
client = MongoClient("mongodb://localhost:27017/")
db = client["twitter_data"]

st.title("Twitter Data scraper")

keyword = st.text_input("Enter keyword or hashtag to search:")
start_date = st.date_input("Enter the start date:")
end_date =st.date_input("Ender end date:")
tweet_limit = st.number_input("Enter the number of tweets to scrape:",min_value=1,max_value=10000)


#scrape Twitter data and display in streamlit

if st.button("Scrape Twitter Data"):
    data = scrape_twitter_data(keyword, str(start_date), str(end_date), int(tweet_limit), db)
    st.write(data)
    

if st.button("Export Data"):
     data_format = st.selectbox("Select data format:"["CSV","JSON"])  
     filename = st.text_input("Enter filename:")
     export_data(data,filename,data_format)                         
     